In [133]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
#если надоела буква T.,однако есть опасность перечислить не всё:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, LongType, BooleanType

import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.functions import window

import datetime


In [134]:
#стандартное создание спарк-сессии
spark = SparkSession.builder.master("local").\
                appName("Home_Work_py").\
                config("spark.driver.bindAddress", "localhost").\
                config("spark.ui.port", "4040").\
                getOrCreate()

In [135]:
#эксперимент со схемой из практики, эксперимент без схемы тоже был, но заместился схемой
schema = StructType(fields=[
    StructField("firstname", StringType()),
    StructField("middlename", StringType()),
    StructField("surname", StringType()),
    StructField("dateadd", StringType()),
    StructField("sex", StringType()),
    StructField("salary", StringType())
])

In [136]:
data = [ ('Петя','Обертонович','Петров','12-12-2010','M',3000)
        ,('Маша','Михайловна','Мишина','10-09-2000','F',5000)
        ,('Коля','Николаевич','Козлов','15-06-1999','M',4000)
        ,('Вася','Евсеевич','Перегудов','01-01-1954','M',7000)
        ,('Амалия','Известновна','Гессен-Дармштадтская','07-07-1754','F',5000)
        ,('Виктор','Иванович','Фауст','08-03-1984','M',6000)
        ,('Леонид','Давидович','Троцкий','17-07-1895','M',11000)
        ,('Джермен','Михайлович','Гвишиани','27-09-1895','M',15000)
        ,('Давид','Изишлопович','Вишман','25-02-1969','M',9000)
        ,('Эзоп','Васильевич','Филимонов','02-04-1978','M',7000)
]

In [137]:
#создаём спарковский датафрейм
df = spark.createDataFrame(data = data, schema = schema )

In [138]:
#эксперимент без схемы тоже был, но заместился схемой
#data = [ ('Петя','','Петров','12-12-2010','M',3000)
#        ,('Маша','Михайловна','Мишина','10-09-2000','F',5000)
#        ,('Коля','Николаевич','','15-06-1999','M',4000)
#        ,('Вася','','Флакин,'01-01-1954','M',7000)
#        ,('Иоханес','Фри','Монсеньеор','30-05-1754','M',5000)
#]

#columns = ['firstame', 'midlename', 'secondname', 'dob', 'gender', 'salary']
#df = spark.createDataFrame(data = data, schema = columns )


In [139]:
df.count()

10

In [140]:
df.show(10)

+---------+-----------+--------------------+----------+---+------+
|firstname| middlename|             surname|   dateadd|sex|salary|
+---------+-----------+--------------------+----------+---+------+
|     Петя|Обертонович|              Петров|12-12-2010|  M|  3000|
|     Маша| Михайловна|              Мишина|10-09-2000|  F|  5000|
|     Коля| Николаевич|              Козлов|15-06-1999|  M|  4000|
|     Вася|   Евсеевич|           Перегудов|01-01-1954|  M|  7000|
|   Амалия|Известновна|Гессен-Дармштадтская|07-07-1754|  F|  5000|
|   Виктор|   Иванович|               Фауст|08-03-1984|  M|  6000|
|   Леонид|  Давидович|             Троцкий|17-07-1895|  M| 11000|
|  Джермен| Михайлович|            Гвишиани|27-09-1895|  M| 15000|
|    Давид|Изишлопович|              Вишман|25-02-1969|  M|  9000|
|     Эзоп| Васильевич|           Филимонов|02-04-1978|  M|  7000|
+---------+-----------+--------------------+----------+---+------+



In [141]:
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- dateadd: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- salary: string (nullable = true)



## a. Создайте схему будущего фрейма данных.
Схема должна включать следующие атрибуты:

·id - уникальный идентификатор посетителя сайта. Тип – последовательность чисел фиксированной длины. Данное поле не является первичным ключом.

· timestamp – дата и время события в формате unix timestamp.

· type – тип события, значение из списка (факт посещения(visit), клик по визуальному элементу страницы(click), скролл(scroll), перед на другую страницу(move)).

· page_id – id текущей страницы. Тип - последовательность чисел фиксированной длины.

· tag – каждая страница с новостью размечается редакцией специальными тегами, которые отражают тематику конкретной новости со страницы. Возможный список тематик: политика, спорт, медицина и т.д.

· sign – наличие у пользователя личного кабинета. Значения – True/False.

In [142]:
schema_web = StructType(fields=[
    StructField("id", LongType()),
    StructField("timestamp", LongType()),
    StructField("type", StringType()),
    StructField("page_id", LongType()),
    StructField("tag", StringType()),
    StructField("sign", BooleanType())
])

In [176]:
data_web = [ (1, 1667627426, "visit", 101, 'Sport', False)
            ,(1, 1667627826, "scroll", 101, 'Sport', False)
            ,(1, 1667627880, "click", 101, 'Sport', False)
            ,(1, 1667657980, "visit", 102, 'Politics', False)
            ,(1, 1667727980, "click", 102, 'Politics', False)
            ,(1, 1667638180, "visit", 102, 'Politics', False)
            ,(2, 1667678326, "visit", 103, 'Politics', True)
            ,(2, 1667628526, "scroll", 103, 'Politics', True)
            ,(2, 1667628680, "click", 104, 'Politics', True)
            ,(2, 1667628697, "click", 104, 'Politics', True)
            ,(2, 1667628880, "visit", 105, 'Business', True)
            ,(2, 1667708810, "click", 105, 'Business', True)
            ,(3, 1667628980, "visit", 106, 'Sport', False)
            ,(1, 1667927980, "click", 102, 'Politics', False)
            ,(3, 1667629180, "click", 106, 'Sport', False)
            ,(3, 1667629480, "visit", 107, 'Business', False)
            ,(1, 1668627980, "click", 102, 'Politics', False)
            ,(3, 1667629580, "scroll", 107, 'Politics', False)
            ,(4, 1667635550, "click", 101, 'Sport', False)
            ,(3, 1667629580, "scroll", 107, 'Business', False)
            ,(3, 1668629570, "click", 106, 'Sport', False)
            ,(5, 1668639580, "click", 106, 'Sport', False)
            ,(6, 1669629560, "scroll", 101, 'Sports', False)
            ,(7, 1669629590, "click", 106, 'Sport', False)]

In [207]:
# c.       Наполните датафрейм данными.
data_lk =[
    (1000,1,"Иванов Иван Иванович", datetime.datetime(1990,7,5), datetime.datetime(2016,8,15)),
    (1060,3,"Петя Обертонович Петров", datetime.datetime(1995,10,2), datetime.datetime(2010,3,20)),
    (1050,2,"Леонид Давидович Троцкий", datetime.datetime(1895,12,1), datetime.datetime(1917,4,1)),
    (1040,7,"Авдотья Феодосевна Новицкая", datetime.datetime(1965,2,1), datetime.datetime(1997,4,1)),
    (1047,5,"Аполлон Евстафьевич Градский", datetime.datetime(1895,12,1), datetime.datetime(2017,4,1)),
    (1045,6,"Валентин Исакиевич Вермонт", datetime.datetime(1895,12,1), datetime.datetime(2014,5,6)),
    (1041,4,"Клаудерис Спаркович Хайвский", datetime.datetime(1895,12,1), datetime.datetime(2022,8,7)),
]

In [211]:
schema_lk = StructType(fields=[
    StructField("id_cab", LongType()),
    StructField("user_id", LongType()),
    StructField("FIO", StringType()),
    StructField("birthdate", DateType()),
    StructField("cab_regdate", DateType()),
])

In [219]:
# b.       Создайте датафрейм с описанной выше схемой данных.
df_web = spark.createDataFrame(data=data_web, schema = schema_web)
df_web.show()

+---+----------+------+-------+--------+-----+
| id| timestamp|  type|page_id|     tag| sign|
+---+----------+------+-------+--------+-----+
|  1|1667627426| visit|    101|   Sport|false|
|  1|1667627826|scroll|    101|   Sport|false|
|  1|1667627880| click|    101|   Sport|false|
|  1|1667657980| visit|    102|Politics|false|
|  1|1667727980| click|    102|Politics|false|
|  1|1667638180| visit|    102|Politics|false|
|  2|1667678326| visit|    103|Politics| true|
|  2|1667628526|scroll|    103|Politics| true|
|  2|1667628680| click|    104|Politics| true|
|  2|1667628697| click|    104|Politics| true|
|  2|1667628880| visit|    105|Business| true|
|  2|1667708810| click|    105|Business| true|
|  3|1667628980| visit|    106|   Sport|false|
|  1|1667927980| click|    102|Politics|false|
|  3|1667629180| click|    106|   Sport|false|
|  3|1667629480| visit|    107|Business|false|
|  1|1668627980| click|    102|Politics|false|
|  3|1667629580|scroll|    107|Politics|false|
|  4|16676355

In [213]:
# b.       Создайте датафрейм с описанной выше схемой данных.
df_lk = spark.createDataFrame(data=data_lk, schema = schema_lk)
df_lk.show()

+------+-------+--------------------+----------+-----------+
|id_cab|user_id|                 FIO| birthdate|cab_regdate|
+------+-------+--------------------+----------+-----------+
|  1000|      1|Иванов Иван Иванович|1990-07-05| 2016-08-15|
|  1060|      3|Петя Обертонович ...|1995-10-02| 2010-03-20|
|  1050|      2|Леонид Давидович ...|1895-12-01| 1917-04-01|
|  1040|      7|Авдотья Феодосевн...|1965-02-01| 1997-04-01|
|  1047|      5|Аполлон Евстафьев...|1895-12-01| 2017-04-01|
|  1045|      6|Валентин Исакиеви...|1895-12-01| 2014-05-06|
|  1041|      4|Клаудерис Спарков...|1895-12-01| 2022-08-07|
+------+-------+--------------------+----------+-----------+



## d. Решите следующие задачи:

In [220]:
# Вывести топ-5 самых активных посетителей сайта
df_top5 = df_web.groupby("id").count().sort("count" , ascending=False)
df_top5.show(5)
# с ФИО
df_top5_rez = df_top5.join(df_lk, on= (df_top5.id==df_lk.user_id)).select(['FIO'])
df_top5_rez.show(5)

+---+-----+
| id|count|
+---+-----+
|  1|    8|
|  3|    6|
|  2|    6|
|  7|    1|
|  6|    1|
+---+-----+
only showing top 5 rows



+--------------------+
|                 FIO|
+--------------------+
|Авдотья Феодосевн...|
|Валентин Исакиеви...|
|Аполлон Евстафьев...|
|Иванов Иван Иванович|
|Петя Обертонович ...|
+--------------------+
only showing top 5 rows



In [221]:
# посчитать процент посетителей у которых есть ЛК
df_tmp = df_web.select(['id','sign']).distinct()
num_users = df_tmp.count()
print("Всего активных пользователей", num_users)
num_users_lk = df_tmp.select(['id','sign']).where(df_tmp.sign == True).count()
print("Всего у кого есть кабинет", num_users_lk)
print(f"Ответ: процент посетителей, у которых есть ЛК = {num_users_lk/num_users*100:.1f}%" )

Всего активных пользователей 7


Всего у кого есть кабинет 1
Ответ: процент посетителей, у которых есть ЛК = 14.3%


In [222]:
# Вывести топ-5 страниц сайта по показателю общего кол-ва кликов на данной странице
df_web.select(["page_id", "type"]).where(df_web.type == 'click').groupby('page_id').count().sort("count", ascending=False).show(5)

+-------+-----+
|page_id|count|
+-------+-----+
|    106|    4|
|    102|    3|
|    101|    2|
|    104|    2|
|    105|    1|
+-------+-----+



In [223]:
#Добавьте столбец к фрейму данных со значением временного диапазона в рамках суток с размером окна – 4 часа(0-4, 4-8, 8-12 и т.д.)

df_web = df_web.withColumn("interval_num", 
    F.floor( 0.25*F.hour( 
                     F.from_unixtime(df_web.timestamp)
                  ) 
           ) 
)
#добавляем колонки с рассчитанным началом и концом интервала
df_web = df_web.withColumn('int0', 4*df_web.interval_num) #.printSchema() #show()
df_web = df_web.withColumn('int1', df_web.int0+4) #.printSchema() #show()
#сливаем колонки в одну
df_web = df_web.withColumn('event_time', F.concat(df_web.int0, F.lit("-"), df_web.int1)) #.printSchema() #show()
#оставляем только нужное
df_web = df_web.select(["id", "timestamp", "type", "page_id", "tag", "sign", "event_time"])
df_web.show()

+---+----------+------+-------+--------+-----+----------+
| id| timestamp|  type|page_id|     tag| sign|event_time|
+---+----------+------+-------+--------+-----+----------+
|  1|1667627426| visit|    101|   Sport|false|       4-8|
|  1|1667627826|scroll|    101|   Sport|false|       4-8|
|  1|1667627880| click|    101|   Sport|false|       4-8|
|  1|1667657980| visit|    102|Politics|false|     12-16|
|  1|1667727980| click|    102|Politics|false|      8-12|
|  1|1667638180| visit|    102|Politics|false|      8-12|
|  2|1667678326| visit|    103|Politics| true|     16-20|
|  2|1667628526|scroll|    103|Politics| true|       4-8|
|  2|1667628680| click|    104|Politics| true|       4-8|
|  2|1667628697| click|    104|Politics| true|       4-8|
|  2|1667628880| visit|    105|Business| true|       4-8|
|  2|1667708810| click|    105|Business| true|       4-8|
|  3|1667628980| visit|    106|   Sport|false|       4-8|
|  1|1667927980| click|    102|Politics|false|     16-20|
|  3|166762918

In [224]:
# Выведите временной промежуток на основе предыдущего задания, в течение которого было больше всего активностей на сайте.
df_web.groupBy("event_time").count().sort("count", ascending=False).show(1)

+----------+-----+
|event_time|count|
+----------+-----+
|       4-8|   13|
+----------+-----+
only showing top 1 row



## Создайте второй фрейм данных,
который будет содержать информацию о ЛК посетителя сайта со следующим списком атрибутов

  Id – уникальный идентификатор личного кабинета
  
  User_id – уникальный идентификатор посетителя
  
  ФИО посетителя
  
  Дату рождения посетителя
  
  Дата создания ЛК

In [225]:
#сделали в ячейках 159-162 (-161)

In [226]:
# Вывести фамилии посетителей, которые читали хотя бы одну новость про спорт.
# пять активных из таблицы с кликами, только данных чуть не хватило(
df_sport = df_web.select(['id', 'tag']).where(df_web.tag == "Sport").distinct() #groupby("id").count().sort("count" , ascending=False)
df_sport.show(5)
# имена пяти(теоретически) активных пользователей
df_sport = df_sport.join(df_lk, on= (df_sport.id==df_lk.user_id), how='left').select(['FIO'])
df_sport.show()

+---+-----+
| id|  tag|
+---+-----+
|  4|Sport|
|  1|Sport|
|  7|Sport|
|  3|Sport|
|  5|Sport|
+---+-----+



+--------------------+
|                 FIO|
+--------------------+
|Авдотья Феодосевн...|
|Аполлон Евстафьев...|
|Иванов Иван Иванович|
|Петя Обертонович ...|
|Клаудерис Спарков...|
+--------------------+



In [227]:
# Проба на жирненькое *Выведите 10% ЛК,
# у которых максимальная разница между датой создания ЛК и датой последнего посещения.

In [232]:
#текущая дата - для тех, кто зарегистрировался  ,но до сих по не зашел на сайт - у них самый максимальный лаг в посещении
now_tstmp = datetime.datetime.timestamp( datetime.datetime.now() )
print(datetime.datetime.now())
print(now_tstmp)
num_lk_10pc = round(0.1*df_lk.count())

df_delta =\
df_lk.withColumn("cab_regdate", F.unix_timestamp( F.col("cab_regdate") ))\
    .join(
df_web.select(["id","timestamp"]).groupBy("id" ).max()
    , on=(df_web.id == df_lk.user_id)
    , how='left')\
.fillna({'max(timestamp)': now_tstmp})\
.withColumn('delta',F.col('max(timestamp)') - F.col('cab_regdate')  )\
.sort('delta', ascending=False)\
.drop("delta").drop("max(timestamp)")\
.limit(num_lk_10pc)

df_delta.show()

2022-11-29 13:24:02.513933
1669728242.513887


+------+-------+--------------------+----------+-----------+---+-------+
|id_cab|user_id|                 FIO| birthdate|cab_regdate| id|max(id)|
+------+-------+--------------------+----------+-----------+---+-------+
|  1050|      2|Леонид Давидович ...|1895-12-01|-1664755200|  2|      2|
+------+-------+--------------------+----------+-----------+---+-------+



In [229]:
#Вывести топ-5 страниц, которые чаще всего посещают мужчины и топ-5 страниц, которые посещают чаще женщины.
df_lk_zh=df_lk.withColumn("sex", F.substring(df_lk.FIO, -3, 3))\
.filter('sex in ("вна","ева","ова")').drop("sex")

df_lk_zh.show()

+------+-------+---+---------+-----------+
|id_cab|user_id|FIO|birthdate|cab_regdate|
+------+-------+---+---------+-----------+
+------+-------+---+---------+-----------+



In [199]:
df_lk.show()

+------+-------+--------------------+----------+--------------------+
|id_cab|user_id|                 FIO| birthdate|         cab_regdate|
+------+-------+--------------------+----------+--------------------+
|  1000|      1|Иванов Иван Иванович|1990-07-05|java.util.Gregori...|
|  1060|      3|Петя Обертонович ...|1995-10-02|java.util.Gregori...|
|  1050|      2|Леонид Давидович ...|1895-12-01|java.util.Gregori...|
|  1040|      7|Авдотья Феодосевн...|1965-02-01|java.util.Gregori...|
|  1047|      5|Аполлон Евстафьев...|1895-12-01|java.util.Gregori...|
|  1045|      6|Валентин Исакиеви...|1895-12-01|java.util.Gregori...|
|  1041|      4|Клаудерис Спарков...|1895-12-01|java.util.Gregori...|
+------+-------+--------------------+----------+--------------------+

